In [112]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
train_input[0]


  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [4]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
logistic_reg = LogisticRegression()

In [7]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
pred = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [9]:
100*(1-(torch.LongTensor(pred) - test_target).abs().sum()/test_input.size(0))

74.0

## Neural Network

In [184]:
class Net(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, 2)

    def forward(self, x):
        print("======")
        print(x.size())
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        print(x.size())
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        print(x.size())
        x = F.relu(self.fc1(x.view(-1, 640)))
        print(x.size())
        print("======")
        return x

In [185]:
my_cool_net = Net()

In [182]:
train_model(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)

torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size

torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size

torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size

torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size

torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size

torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size

torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size

torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size

torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size

torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size([4, 1, 28, 50])
torch.Size([4, 32, 8, 15])
torch.Size([4, 64, 2, 5])
torch.Size([4, 2])
torch.Size

In [178]:
nb_errors = compute_nb_errors(my_cool_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)

In [179]:
nb_errors/test_input.size(0)

0.51